In [1]:
# initialized based on colab code but run locally 

path_prefix = "C:/Users/tania/OneDrive - TM-IT Consulting Inc/Documents/4th yr/spring25/cmpt410/CMPT 726-410 Spring 2025 A2 starter code/"

import os
import sys
sys.path.insert(1, path_prefix)

print(os.listdir(path_prefix))

['a2_classifier.ipynb', 'a2_siamese_net.ipynb', 'data', 'utils.py', '__pycache__']


In [2]:
from utils import *
import random

torch.manual_seed(410)
np.random.seed(410)
random.seed(410)

In [3]:
transform = transforms.Compose([transforms.ToTensor()])
fashion_mnist_train = datasets.FashionMNIST(root=path_prefix + 'data/', train=True, download=True, transform=transform)
fashion_mnist_val = datasets.FashionMNIST(root=path_prefix + 'data/', train=False, download=True, transform=transform)

# Customizing FashionMNIST dataset for the siamese network
train_dataset = CustomFashionMNIST(fashion_mnist_train)
val_dataset = CustomFashionMNIST(fashion_mnist_val)


In [4]:
class SiameseNet(AbstractSiameseNet):
    def __init__(self, input_size, output_size, layer_width=8):
        super(SiameseNet, self).__init__()
        self.layers = nn.Sequential(
            nn.Flatten(),
            # Define the layers of the Siamese network. 
            # The model is best model found in part 3(d)
            nn.Linear(input_size, 187), # hidden layer with 187 neurons
            nn.ReLU(), # and ReLU activation
            nn.Linear(187, 64), # hidden layer with 64 neurons
            nn.ReLU(), # and ReLU activation
            nn.Linear(64, 32), # hidden layer with 32 neurons
            nn.ReLU(), # and ReLU activation
            nn.Linear(32, output_size) # output layer  
        )
    
    def forward(self, x1, x2):
        # Define the forward pass of the Siamese network
        out1 = self.layers(x1)
        out2 = self.layers(x2)

        return out1, out2



In [5]:
# Model initialization and summary
model = SiameseNet(input_size=28*28, output_size=10)
summary(model.cpu(), [(1, 28, 28), (1, 28, 28)])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 187]         146,795
              ReLU-3                  [-1, 187]               0
            Linear-4                   [-1, 64]          12,032
              ReLU-5                   [-1, 64]               0
            Linear-6                   [-1, 32]           2,080
              ReLU-7                   [-1, 32]               0
            Linear-8                   [-1, 10]             330
           Flatten-9                  [-1, 784]               0
           Linear-10                  [-1, 187]         146,795
             ReLU-11                  [-1, 187]               0
           Linear-12                   [-1, 64]          12,032
             ReLU-13                   [-1, 64]               0
           Linear-14                   

In [6]:
# Configure hyperparameters 
LEARNING_RATE = 0.001 # the learning rate which achieves the best performance from the previous experiments
EPOCHS = 10
BATCH_SIZE = 64
OPTIMIZER = torch.optim.Adam(model.parameters(), weight_decay=0, lr = LEARNING_RATE) # optimizer which achieves the best performance from the previous experiments
loss_function = ContrastiveLoss() # Contrastive loss minimizes the distance between similar pairs and maximizes the distance between dissimilar pairs

In [7]:
# Training the model
trainer = SiameseTrainer(model, train_dataset, val_dataset, loss_function, BATCH_SIZE, EPOCHS, OPTIMIZER, path_prefix)

train_loss, validation_loss, train_accuracy, validation_accuracy = trainer.train_loop()
trainer.plot_losses(train_loss, validation_loss)
trainer.plot_accuracies(train_accuracy, validation_accuracy)

  0%|          | 0/938 [00:00<?, ?it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()